In [1]:
from cascade.benchmarks.reporting import logParse
import pandas as pd
from bokeh.io import curdoc, show
from bokeh.models import ColumnDataSource, Grid, HBar, LinearAxis, Plot, VSpan
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [34]:
def plotTaskLane(data: dict, scale, width):
    Td = data['task_durations']
    nc = ['planned', 'enqueued', 'started', 'loaded', 'computed', 'published', 'completed']
    df = (Td[nc]/scale).astype(int)
    zero = df.planned.min()
    df = df - zero
    df = df.assign(worker=Td.worker)

    workerToLane = {e: i for i, e in enumerate(Td.worker.drop_duplicates().sort_values())}
    print(workerToLane) # TODO use the labels in the if instead

    plot = Plot(
        title=None, width=width, height=500,
        min_border=0, toolbar_location=None,
    ) # TODO derive width from data

    def boxTask(left, right, color):
        source = ColumnDataSource(dict(y=df.worker.map(workerToLane), left=left, right=right))    
        glyph = HBar(y="y", right="right", left="left", height=0.5, fill_color=color)
        plot.add_glyph(source, glyph)

    def boxTransmit(worker, left, right, color):
        source = ColumnDataSource(dict(y=worker.map(workerToLane) - 0.5, left=left, right=right))    
        glyph = HBar(y="y", right="right", left="left", height=0.25, fill_color=color)
        plot.add_glyph(source, glyph)        
        
    def barController(df, action, color):
        df = df.query(f"action == '{action}'")
        df = (df[['at']] / scale).astype(int) - zero
        df = df.assign(width=2)
        source = ColumnDataSource(dict(x=df['at'], width=df.width))
        glyph = VSpan(x="x", line_width="width", line_color=color)
        plot.add_glyph(source, glyph)

    Ctrl = data['controller']
    barController(Ctrl, "plan", '#00aa00')
    barController(Ctrl, "act", '#0000aa')
    barController(Ctrl, "wait", '#aa0000')
    barController(Ctrl, "shutdown", '#000000')
    
    boxTask(df.planned, df.enqueued, '#111111')
    boxTask(df.enqueued, df.started, '#dd1111')
    boxTask(df.started, df.loaded, '#1111dd')
    boxTask(df.loaded, df.computed, '#11dd11')
    boxTask(df.computed, df.published, '#1111dd')
    boxTask(df.published, df.completed, '#444444')

    Rd = data['transmit_durations']
    nc = ['planned', 'started', 'loaded', 'received', 'unloaded', 'completed']
    df = (Rd[nc]/scale).astype(int)
    df = df - zero
    df = df.assign(target=Rd.target, source=Rd.source)
    boxTransmit(df.target, df.planned, df.received, '#111111') # target waiting
    boxTransmit(df.target, df.received, df.unloaded, '#1111dd') # target memcpy
    boxTransmit(df.target, df.unloaded, df.completed, '#444444') # target callback
    boxTransmit(df.source, df.planned, df.started, '#111111') # ctrl2source comm delay
    boxTransmit(df.source, df.started, df.loaded, '#1111dd') # source memcpy
    boxTransmit(df.source, df.loaded, df.received, '#444444') # source memcpy + network
    
    xaxis = LinearAxis()
    plot.add_layout(xaxis, 'below')
    
    yaxis = LinearAxis()
    plot.add_layout(yaxis, 'left')
    
    plot.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
    plot.add_layout(Grid(dimension=1, ticker=yaxis.ticker))
    
    curdoc().add_root(plot)
    
    return show(plot)

In [87]:
l4prob = logParse(["l4prob.txt"])
plotTaskLane(l4prob, 1e6, 2000)

{'fiab:w3': 0, 'fiab:w0': 1, 'fiab:w2': 2, 'fiab:w1': 3}


In [41]:
l2_2prob = logParse(["l2_2.prob.txt"])
plotTaskLane(l2_2prob, 1e6, 2000)

{'h0:w0': 0, 'h0:w1': 1, 'h1:w0': 2, 'h1:w1': 3}


In [88]:
l1_4all = logParse(["l1_4.all.txt"])
plotTaskLane(l1_4all, 1e6, 6000)

{'h0:w3': 0, 'h0:w0': 1, 'h0:w2': 2, 'h0:w1': 3}


In [40]:
l2_2all = logParse(["l2_2.all.txt"])
plotTaskLane(l2_2all, 1e6, 6000)

{'h0:w0': 0, 'h0:w1': 1, 'h1:w0': 2, 'h1:w1': 3}


In [38]:
h1all = logParse(["h1.all.txt"])
plotTaskLane(h1all, 1e6, 6000)

{'h1:w0': 0, 'h1:w1': 1, 'h1:w2': 2, 'h1:w3': 3, 'h1:w4': 4, 'h1:w5': 5, 'h1:w6': 6, 'h1:w7': 7, 'h1:w8': 8, 'h1:w9': 9, 'h2:w0': 10, 'h2:w1': 11, 'h2:w2': 12, 'h2:w3': 13, 'h2:w4': 14, 'h2:w5': 15, 'h2:w6': 16, 'h2:w7': 17, 'h2:w8': 18, 'h2:w9': 19}


In [39]:
h2all = logParse(["h2.all.txt"])
plotTaskLane(h2all, 1e6, 10000)

{'h1:w0': 0, 'h1:w1': 1, 'h1:w2': 2, 'h1:w3': 3, 'h1:w4': 4, 'h1:w5': 5, 'h1:w6': 6, 'h1:w7': 7, 'h1:w8': 8, 'h1:w9': 9, 'h2:w0': 10, 'h2:w1': 11, 'h2:w2': 12, 'h2:w3': 13, 'h2:w4': 14, 'h2:w5': 15, 'h2:w6': 16, 'h2:w7': 17, 'h2:w8': 18, 'h2:w9': 19, 'h3:w0': 20, 'h3:w1': 21, 'h3:w2': 22, 'h3:w3': 23, 'h3:w4': 24, 'h3:w5': 25, 'h3:w6': 26, 'h3:w7': 27, 'h3:w8': 28, 'h3:w9': 29, 'h4:w0': 30, 'h4:w1': 31, 'h4:w2': 32, 'h4:w3': 33, 'h4:w4': 34, 'h4:w5': 35, 'h4:w6': 36, 'h4:w7': 37, 'h4:w8': 38, 'h4:w9': 39}
